**KNN (k nearest neighbor)**

Aprendizado baseado em instâncias. Para determinar classe a qual pertence um novo dado o algoritmo procura traçar uma “distância” entre este novo ponto e os seus K vizinhos mais próximos para fazer a classificação.


Características :

O por ser baseado em instâncias o  KNN é um algoritmo que não precisa de treinamento (e portanto não gera um modelo). Quando é chamado para fazer uma  previsão ou classificação o algoritmo busca os dados do dataset de “treino” e é executado de maneira lazy.




In [85]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.compose import make_column_transformer
from sklearn.neighbors import KNeighborsClassifier


# carregando a base de dados de censo
base = pd.read_csv('../../res/census.csv')
base.shape



(32561, 15)

In [86]:
# separando os dados de previsao e classificacao
previsores = base.iloc[:, 0:14].values
classificadores = base.iloc[:, 14].values

#gerando uma copia dos dados originais para fazer mais testes abaixo
previsores_escalonados=previsores.copy()


#efetuando correcoes nos dados do censo

#transformando dados categorios da base em dados discretos

labelencoder_prev = LabelEncoder()
previsores[:, 1] = labelencoder_prev.fit_transform(previsores[:, 1])
previsores[:, 3] = labelencoder_prev.fit_transform(previsores[:, 3])
previsores[:, 5] = labelencoder_prev.fit_transform(previsores[:, 5])
previsores[:, 6] = labelencoder_prev.fit_transform(previsores[:, 6])
previsores[:, 7] = labelencoder_prev.fit_transform(previsores[:, 7])
previsores[:, 8] = labelencoder_prev.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_prev.fit_transform(previsores[:, 9])
previsores[:, 13] = labelencoder_prev.fit_transform(previsores[:, 13])



In [87]:
# padronizando os valores nao discretos da copia dos previsores ( nao deve ser feito para todos os parametros sob risco de degradar a precisao do algoritimo)

#o min max scaler foi mais interessante para este caso
#scaler = MinMaxScaler(feature_range=(0, 1))
scaler = StandardScaler()

#transformando dados categorios da copia da base em dados discretos
labelencoder_prev = LabelEncoder()
previsores_escalonados[:, 1] = labelencoder_prev.fit_transform(previsores_escalonados[:, 1])
previsores_escalonados[:, 3] = labelencoder_prev.fit_transform(previsores_escalonados[:, 3])
previsores_escalonados[:, 5] = labelencoder_prev.fit_transform(previsores_escalonados[:, 5])
previsores_escalonados[:, 6] = labelencoder_prev.fit_transform(previsores_escalonados[:, 6])
previsores_escalonados[:, 7] = labelencoder_prev.fit_transform(previsores_escalonados[:, 7])
previsores_escalonados[:, 8] = labelencoder_prev.fit_transform(previsores_escalonados[:, 8])
previsores_escalonados[:, 9] = labelencoder_prev.fit_transform(previsores_escalonados[:, 9])
previsores_escalonados[:, 13] = labelencoder_prev.fit_transform(previsores_escalonados[:, 13])


print("\nVisualizando estatisticas dos dados nao discretos antes do escalonamento\n")
for x in range(3):
    print('coluna ',x,"\n")
    print(previsores_escalonados[:,[4,10,12]][x].min())
    print(previsores_escalonados[:,[4,10,12]][x].max())
    print(previsores_escalonados[:,[4,10,12]][x].mean())
    print(previsores_escalonados[:,[4,10,12]][x].var())
    print("\n")


#escalonando os previsores da copia dos dados
previsores_escalonados = scaler.fit_transform(previsores_escalonados)

print("\nVisualizando estatisticas dos dados nao discretos depois do escalonamento\n")
for x in range(3):
    print('coluna ',x,"\n")
    print(previsores_escalonados[:,[4,10,12]][x].min())
    print(previsores_escalonados[:,[4,10,12]][x].max())
    print(previsores_escalonados[:,[4,10,12]][x].mean())
    print(previsores_escalonados[:,[4,10,12]][x].var())
    print("\n")
    
labelencoder_classe = LabelEncoder()
classificadores = labelencoder_classe.fit_transform(classificadores)



#separando os valores de teste e treinamento  para os previsores escalonados e nao escalonados
previsores_treinamento, previsores_teste, classificadores_treinamento1, classificadores_teste1 = train_test_split(previsores, classificadores, test_size=0.15, random_state=0)
previsores_escalonados_treinamento, previsores_escalonados_teste, classificadores_treinamento, classificadores_teste = train_test_split(previsores_escalonados, classificadores, test_size=0.15, random_state=0)




Visualizando estatisticas dos dados nao discretos antes do escalonamento

coluna  0 

13
2174
742.3333333333334
1024956.222222222


coluna  1 

0
13
8.666666666666666
37.55555555555556


coluna  2 

0
40
16.333333333333332
293.5555555555556



Visualizando estatisticas dos dados nao discretos depois do escalonamento

coluna  0 

-0.035429446972779874
1.1347387637961852
0.4159207373469685
0.26398513020240394


coluna  1 

-2.222153121346444
1.1347387637961852
-0.4111116137030442
1.9132836891763105


coluna  2 

-0.42005962401595
-0.035429446972779874
-0.20046985151586794
0.026144545620963824




In [88]:
# instanciando o naive bayes com o scikit
classificador = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2,n_jobs=2)
classificador.fit(previsores_escalonados_treinamento, classificadores_treinamento)

# rodando previsoes com o dados de teste (copia)
previsoes_dados_escalonados = classificador.predict(previsores_escalonados_teste)

# fazendo o fit com os dados normais
classificador.fit(previsores_treinamento, classificadores_treinamento1)
previsoes = classificador.predict(previsores_teste)


In [89]:
#testes dessa instancia algoritimo

# o dado de precisao per se nao quer dizer muita coisa e preciso verificar outras metricas
precisao_escalonados = accuracy_score(classificadores_teste, previsoes_dados_escalonados)
precisao = accuracy_score(classificadores_teste1, previsoes)

# uma dessas metricas eh a matriz de confusao ... ela e capaz de mostrar o desempenho do algoritimo para cada classe 
matriz_escalonados = confusion_matrix(classificadores_teste, previsoes_dados_escalonados)
matriz = confusion_matrix(classificadores_teste1, previsoes)

#o scikit tambem possui uma classe utilitaria que prove um report mais detalhado...
report_escalonados = classification_report(classificadores_teste, previsoes_dados_escalonados)
report = classification_report(classificadores_teste1, previsoes)

print("Precisão dados normais / escalonados :\n")
print(precisao,'/',precisao_escalonados)
print("\nMatriz de confusão dados normais / escalonados:\n")
print(matriz)
print("\n")
print(matriz_escalonados)
print("\nReport dados normais / escalonados:\n")
print (report)
print("\n")
print (report_escalonados)

Precisão dados normais / escalonados :

0.7746161719549641 / 0.8219037871033776

Matriz de confusão dados normais / escalonados:

[[3406  287]
 [ 814  378]]


[[3331  362]
 [ 508  684]]

Report dados normais / escalonados:

              precision    recall  f1-score   support

           0       0.81      0.92      0.86      3693
           1       0.57      0.32      0.41      1192

    accuracy                           0.77      4885
   macro avg       0.69      0.62      0.63      4885
weighted avg       0.75      0.77      0.75      4885



              precision    recall  f1-score   support

           0       0.87      0.90      0.88      3693
           1       0.65      0.57      0.61      1192

    accuracy                           0.82      4885
   macro avg       0.76      0.74      0.75      4885
weighted avg       0.82      0.82      0.82      4885



<br>**TODO :Verificar a base de treinamento para melhorar a distribuicao das classes e verificar se ha alguma melhora**